In [1]:
import pandas as pd

In [100]:
import os
from pathlib import Path

import pandas as pd

from asme.data.datamodule.converters import CsvConverter
from asme.data.datamodule.util import read_csv


In [102]:
input_dir = Path("/Users/lisa/recommender/datasets/ml-1m-raw")
output_file = Path("/Users/lisa/recommender/datasets/ml-1m-extended/ml-1m-extendend.csv")

In [101]:
base_config= "/Users/lisa/recommender/src/asme.plp_paper/local-ml-1m.yaml"
expanded_config= "/Users/lisa/recommender/src/asme.plp_paper/local-ml-1m-plp.yaml"

## Download and split Dataset with Default Values

In [4]:
from asme.core.utils.run_utils import load_config, create_container
import typer
import torch

torch.multiprocessing.set_start_method('spawn', force=True)

def create_original_data(config_file: Path = typer.Argument(..., help='the path to the config file', exists=True)) -> None:
    config_file_path = Path(config_file)
    config = load_config(config_file_path)
    #Creates the dataset:
    container = create_container(config)

create_original_data(Path(base_config))

Preprocessing dataset:
Unpacking dataset.
Applying preprocessing step 'Converting to CSV' (1/2)
Skipping this step since dry run is available.
Applying preprocessing step 'Generating ratio split. (0.8/0.1/0.1).' (2/2)
Property 'features.item.tokenizer.vocabulary.file' was not found in the configuration. Inferring it to '/Users/lisa/recommender/datasets/ml-1m/ml-1m/ratio_split-0.8_0.1_0.1/ml-1m.vocabulary.title.txt'.
Your module contains a submodule named <embedding> that does not report its hyperparameters, consider adding @save_hyperparameters annotation.


File /Users/lisa/recommender/datasets/ml-1m/ml-1m/ml-1m.zip already downloaded


In [122]:
input_dir = Path("/Users/lisa/recommender/datasets/ml-1m/ml-1m/ratio_split-0.8_0.1_0.1")
output_dir = Path("/Users/lisa/recommender/datasets/ml-1m/ml-1m-overview/ratio_split-0.8_0.1_0.1")
name = "ml-1m"

import os
def create_extended_movielens_data(input_dir, output_dir, name, stage):
    file_type = ".csv"
    encoding = "latin-1"
    delimiter = "\t"
    item_df = read_csv(input_dir, f'{name}.{stage}', file_type, "\t", header=0, encoding=encoding)
    item_df["cat_title"] = item_df["title"]
    item_df["old_title"] = item_df["title"]
    item_df["item_id_type"] = 1
    page_df_mod = item_df.copy()
    page_df_mod["old_title"] = page_df_mod["title"]
    page_df_mod["title"] = "OVERVIEW-PAGE"
    page_df_mod["cat_title"] = page_df_mod["genres"]
    page_df_mod["item_id_type"] = 0
    page_df_mod["rating"] = -1
    page_df_mod["year"] = 0
    item_df['original_order'] = item_df.groupby(['userId', 'timestamp']).cumcount() + 1
    page_df_mod['original_order'] = page_df_mod.groupby(['userId', 'timestamp']).cumcount() + 1
    item_df = pd.concat([item_df,page_df_mod], ignore_index=True)
    item_df = item_df.sort_values(["userId","timestamp","original_order","item_id_type"])
    item_df = item_df[['userId', 'rating', 'timestamp', 'gender', 'age',
                       'occupation', 'title', 'genres', 'year', 'user_all', 'cat_title', 'item_id_type']]
    os.makedirs(output_dir, exist_ok=True)
    item_df.to_csv(f'{output_dir}/{name}.{stage}{file_type}', sep=delimiter, index=False)

for stage in ["test","train","validation"]:
    create_extended_movielens_data(input_dir,output_dir,name,stage)
